# Import Libraries

In [ ]:
import tensorflow as tf
import numpy as np
import cv2

# Load the model
new_model = tf.keras.models.load_model('64BResNetmodel.keras')

2.16.2


## Live Detection with OpenCV 

In [ ]:
# Path to the Haar Cascade file for face detection
path = '/haarcascade_frontalface_default.xml'
font_scale = 1.5
font = cv2.FONT_HERSHEY_PLAIN

# Define the background color for the rectangle
rectangle_bgr = (255, 255, 255)
# Create a black image
img = np.zeros((500, 500))
# Text to be displayed
text = "Some text in a box!"
# Get the width and height of the text box
(text_width, text_height) = cv2.getTextSize(text, font, fontScale=font_scale, thickness=1)[0]
# Set the text offset
text_offset_x = 10
text_offset_y = img.shape[0] - 25
# Set the box coordinates
box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height - 2))
# Draw the rectangle on the image
cv2.rectangle(img, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)
# Put the text on the image
cv2.putText(img, text, (text_offset_x, text_offset_y), font, fontScale=font_scale, color=(0, 0, 0), thickness=1)

# Capture video from the webcam
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# List of emotion labels
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

while True:
    # Read a frame from the webcam
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame")
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    face_roi = None  # Initialize face_roi to avoid issues if no faces are found

    for x, y, w, h in faces:
        # Get the region of interest in the grayscale and color frames
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        # Detect faces in the region of interest
        facess = face_cascade.detectMultiScale(roi_gray)
        if len(facess) == 0:
            print("Face not detected in region of interest")
        else:
            for (ex, ey, ew, eh) in facess:
                face_roi = roi_color[ey: ey+eh, ex: ex+ew]

    # Proceed only if a face was detected
    if face_roi is not None:
        # Resize the face region to the required size
        final_image = cv2.resize(face_roi, (224, 224))
        final_image = np.expand_dims(final_image, axis=0)
        final_image = final_image / 255.0

        # Predict the emotion using the loaded model
        Predictions = new_model.predict(final_image)
        status = emotion_labels[np.argmax(Predictions)]

        # Draw the status on the frame
        x1, y1, w1, h1 = 0, 0, 175, 75
        cv2.rectangle(frame, (x1, y1), (x1 + w1, y1 + h1), (0, 0, 0), -1)
        cv2.putText(frame, status, (x1 + int(w1/10), y1 + int(h1/2)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, status, (100, 150), font, 3, (0, 0, 255), 2, cv2.LINE_4)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255))

    # Display the frame with the emotion recognition
    cv2.imshow('Face Emotion Recognition', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!
[ WARN:0@847.292] global cap.cpp:323 open VIDEOIO(OBSENSOR): raised unknown C++ exception!




[12/24 11:26:18.618801][info][104692][Context.cpp:69] Context created with config: default config!
[12/24 11:26:18.618823][info][104692][Context.cpp:74] Context work_dir=/Users/measheanhsengheng/Downloads/Face_Emotion_Recognition
[12/24 11:26:18.618841][info][104692][Context.cpp:77] 	- SDK version: 1.9.4
[12/24 11:26:18.618843][info][104692][Context.cpp:78] 	- SDK stage version: main
[12/24 11:26:18.618855][info][104692][Context.cpp:82] get config EnumerateNetDevice:false
[12/24 11:26:18.618858][info][104692][MacPal.cpp:36] createObPal: create MacPal!
[12/24 11:26:18.622774][info][104692][MacPal.cpp:104] Create PollingDeviceWatcher!
[12/24 11:26:18.622786][info][104692][DeviceManager.cpp:15] Current found device(s): (0)
[12/24 11:26:18.622791][info][104692][Pipeline.cpp:15] Try to create pipeline with default device.
[12/24 11:26:18.622794][warning][104692][ObException.cpp:5] No device found, fail to create pipeline!
[12/24 11:26:18.623156][info][104692][Context.cpp:90] Context destroy

KeyboardInterrupt: 

: 